<a href="https://colab.research.google.com/github/pascalghanimi/Ski-Classification-AI/blob/main/Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Datenvisualisierung
import pickle
import pandas as pd

file_path = "features_with_labels.pkl"

with open(file_path, "rb") as pkl_file:
  features_with_labels_dic = pickle.load(pkl_file)
print(features_with_labels_dic)

df = pd.DataFrame({
    "Frame": list(features_with_labels_dic["COM_to_ground"].keys()),
    "COM_to_ground": list(features_with_labels_dic["COM_to_ground"].values()),
    "Label": [features_with_labels_dic.get(frame, 0) for frame in features_with_labels_dic["COM_to_ground"].keys()]
})

df.head()

{'COM_to_ground': {0: 0.83516055, 1: 0.83531225, 2: 0.8258441, 3: 0.8233454, 4: 0.8134174, 5: 0.808589, 6: 0.809546, 7: 0.8108446, 8: 0.8079058, 9: 0.80322814, 10: 0.80321217, 11: 0.7997785, 12: 0.788425, 13: 0.7929602, 14: 0.78522795, 15: 0.7817862, 16: 0.7795192, 17: 0.7715101, 18: 0.7714118, 19: 0.7654959, 20: 0.7735648, 21: 0.7695321, 22: 0.7670833, 23: 0.7636223, 24: 0.7623254, 25: 0.75666046, 26: 0.7539136, 27: 0.75270075, 28: 0.75509053, 29: 0.75143194, 30: 0.75414115, 31: 0.7570532, 32: 0.76489687, 33: 0.7735289, 34: 0.7897625, 35: 0.7994213, 36: 0.8149826, 37: 0.82652134, 38: 0.8428307, 39: 0.8455673, 40: 0.8630697, 41: 0.87122834, 42: 0.8809966, 43: 0.8800359, 44: 0.8854804, 45: 0.8947362, 46: 0.889527, 47: 0.9010812, 48: 0.90366524, 49: 0.90048194, 50: 0.90168905, 51: 0.8974404, 52: 0.89560395, 53: 0.8942573, 54: 0.88930565, 55: 0.88446933, 56: 0.8838185, 57: 0.88569283, 58: 0.88442343, 59: 0.8735657, 60: 0.86955655, 61: 0.86682963, 62: 0.8604068, 63: 0.8600429, 64: 0.861037

,Frame,COM_to_ground,Label
0,0,0.835161,0
1,1,0.835312,0
2,2,0.825844,0
3,3,0.823345,0
4,4,0.813417,0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import pandas as pd
import numpy as np

# 🔹 Datei laden
file_path = "features_with_labels.pkl"

with open(file_path, "rb") as pkl_file:
    features_with_labels_dic = pickle.load(pkl_file)

# 🔹 DataFrame erstellen
df = pd.DataFrame({
    "Frame": list(features_with_labels_dic["COM_to_ground"].keys()),
    "COM_to_ground": list(features_with_labels_dic["COM_to_ground"].values()),
    "Label": [features_with_labels_dic["Schwung_labels"].get(frame, 0) for frame in features_with_labels_dic["COM_to_ground"].keys()]
})

# 🔹 LSTM Input vorbereiten: **GANZE SEQUENZ (117 Frames)**
X = df["COM_to_ground"].values.reshape(1, -1, 1)  # Shape (Batch=1, Time=117, Features=1)
Y = df["Label"].values.reshape(1, -1)  # Shape (Batch=1, Time=117)

# In Torch-Tensor umwandeln
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.long)

# 🔹 LSTM-Modell definieren
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Klassifikation für jedes Frame

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out)  # Kein [:, -1, :] mehr, da jedes Frame eine Vorhersage braucht
        return out

# 🔹 Modell initialisieren
input_size = 1  # Nur COM_to_ground
hidden_size = 64
num_layers = 2
output_size = 3  # 0 = Kein Schwung, 1 = Linksschwung, 2 = Rechtsschwung

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 🔹 Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 🔹 Training
num_epochs = 50

for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(0), Y_train.squeeze(0))  # Loss berechnen
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training abgeschlossen! 🚀")


Epoch 0, Loss: 1.1117448806762695
Epoch 10, Loss: 0.9717342853546143
Epoch 20, Loss: 0.6892607808113098
Epoch 30, Loss: 0.6544957160949707
Epoch 40, Loss: 0.6317602396011353
Training abgeschlossen! 🚀


In [3]:
with torch.no_grad():  # Keine Gradientenberechnung nötig
    predictions = model(X_train)
    predicted_classes = torch.argmax(predictions, dim=2)  # Klasse mit höchster Wahrscheinlichkeit
print(predicted_classes)


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


In [ ]:
correct = (predicted_classes.squeeze(0) == Y_train.squeeze(0)).sum().item()
accuracy = correct / len(Y_train.squeeze(0))
print(f"Modellgenauigkeit: {accuracy:.2f}")


Modellgenauigkeit: 0.66
